## 1. Tạo dataset lưu path của ảnh cần test



In [32]:
# import os
# import csv
# base_dir = "C:/Users/Admin/Desktop/IMG_Test"
# with open('image_paths.csv', 'w', newline='', encoding='utf-8') as csvfile:
#     # Tạo một đối tượng DictWriter
#     fieldnames = ['name', 'face', 'unknown','is_same']  # Tên trường đã được sửa
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     # Ghi tên cột
#     writer.writeheader()
#     # Duyệt qua tất cả các thư mục trong thư mục cơ sở
#     for name in os.listdir(base_dir):
#         name_path = os.path.join(base_dir, name)
#         if os.path.isdir(name_path):
#             # Lấy đường dẫn của các hình ảnh trong thư mục hiện tại
#             image_files = os.listdir(name_path)
#             # Giả sử cặp ảnh luôn là các tệp tin có cùng chỉ số trong danh sách
#             for i in range(0, len(image_files), 2):
#                 face_path = os.path.join(name_path, image_files[i])
#                 unknown_path = os.path.join(name_path, image_files[i+1])
#                 # Ghi đường dẫn của cặp ảnh vào tập tin CSV
#                 writer.writerow({'name': name, 'face': face_path, 'unknown': unknown_path})
# print("File CSV đã được tạo thành công.")


File CSV đã được tạo thành công.


## 2. Đọc ảnh từ file dataset, tiến hành so sánh sử dụng face_recognition


In [2]:
import os
import pandas as pd
import face_recognition

# Hàm kiểm tra khuôn mặt trong hai ảnh và trả về 1 nếu trùng khớp, ngược lại trả về 0
def compare_faces(face_image_path, unknown_image_path):
    # Load ảnh khuôn mặt đã biết
    known_image = face_recognition.load_image_file(face_image_path)
    # Load ảnh chưa biết
    unknown_image = face_recognition.load_image_file(unknown_image_path)
    # Tìm tất cả các khuôn mặt trong cả hai ảnh
    known_face_encodings = face_recognition.face_encodings(known_image)
    unknown_face_encodings = face_recognition.face_encodings(unknown_image)
    # So sánh khuôn mặt
    for known_face_encoding in known_face_encodings:
        for unknown_face_encoding in unknown_face_encodings:
            # So sánh các khuôn mặt
            results = face_recognition.compare_faces([known_face_encoding], unknown_face_encoding)
            if results[0]:
                return 1
    return 0

# Đường dẫn đến tập tin CSV chứa đường dẫn của các ảnh
csv_file = 'image_paths.csv'

# Đọc tập tin CSV bằng Pandas
df = pd.read_csv(csv_file)

# Tạo một mảng để lưu kết quả
results = []

# Duyệt qua từng hàng trong DataFrame và gọi hàm compare_faces để kiểm tra khuôn mặt
for index, row in df.iterrows():
    result = compare_faces(row['face'], row['unknown'])
    results.append(result)

results


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1]

## 3. Lấy trường is_same từ file dataset


In [5]:
import numpy as np
y = df.iloc[:,-1].values
np.count_nonzero(y==0)

35

In [9]:
comp = pd.DataFrame({"y" : y, "y_preds" : results })
comp

,y,y_preds
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
101,0,0
102,0,1
103,1,1
104,0,0


## 4. Đánh giá kết quả


In [10]:
different_pairs = comp[comp['y'] != comp['y_preds']]
different_pairs

,y,y_preds
19,0,1
23,0,1
33,0,1
38,0,1
41,0,1
63,1,0
66,1,0
69,0,1
100,0,1
102,0,1


In [11]:
import chart1
TP, FP, TN, FN = chart1.index(comp)
chart = chart1.Macro_Average("Face_recognition",chart1.Percision(TP,FP),chart1.Recall(TP,FN),chart1.F1_score(TP,FP,FN),chart1.Accuracy(TP,TN,FP,FN))
chart

,Model,Precision,Recall,F1_score,Accuracy
0,Face_recognition,89.61%,97.18%,93.24%,90.57%


- Percision: Thể hiện sự chuẩn xác của việc phát hiện các điểm Positive. Chỉ số này càng cao thì Model nhận các điểm Positive càng chuẩn.
- Recall: Thể hiện khả năng phát hiện tất cả các Positive, tỉ lệ này càng cao thì cho thấy khả năng bỏ qua các điểm Positive càng thấp.
- F1 score: Là số dung hòa giữa Recall và Percision
- Accuracy: tỷ lệ giữa số lượng các dự đoán đúng (true positives và true negatives - TP + TN) và tổng số mẫu dữ liệu. Accuracy đo lường tỷ lệ dự đoán đúng trên toàn bộ tập dữ liệu

In [2]:
import matplotlib.pyplot as plt

# Tạo ma trận nhầm lẫn từ các thông số TP, FP, TN, FN
conf_matrix = [[TP, FP],
            [FN, TN]]

# Vẽ ma trận nhầm lẫn
plt.imshow(conf_matrix, cmap=plt.cm.Blues, interpolation='nearest')
plt.title('Confusion Matrix')
plt.colorbar()

# Đặt nhãn cho trục x và trục y
classes = ['1', '0']
plt.xticks([0, 1], classes)
plt.yticks([0, 1], classes)

# Hiển thị số lượng mẫu trong từng ô
thresh = max(max(conf_matrix)) / 2.  
for i in range(len(classes)):
    for j in range(len(classes)):
        plt.text(j, i, str(conf_matrix[i][j]), horizontalalignment="center",
                 color="white" if conf_matrix[i][j] > thresh else "black")

plt.ylabel('Predicted label')
plt.xlabel('True label')
plt.tight_layout()
plt.show()


NameError: name 'TP' is not defined

In [17]:
d = {0,1,2}
for x in d:
    print(d.add(x))
print(d)

None
None
None
{0, 1, 2}
